# Panel Loading

Notebook to load panel data into the databse

## Imports

In [1]:
import psycopg2
from tabulate import tabulate
#import a folder in the parent directory
import sys
sys.path.append('../')
import dbtools.dbtools as qrs
import dbtools.load as load

## Connection

In [2]:
try:
    # Connect to the PostgreSQL database
    conn = qrs.connect()
    print("Connected to the database")

except (Exception, psycopg2.DatabaseError) as error:
    print(error)

Connected to the database


## Material id search

A panel is made of a material, so the panel table is linked to the material table.

Here we have to check the material table and to set the material of the panel.

The `load_panel` function now includes the `layer_layout` parameter which was moved from the `load_material` function. This parameter allows you to specify the orientation of layers in the panel as a list of integers representing angles.

For example: `layer_layout = [0, 45, 90, -45]` represents a 4-layer panel with layers oriented at 0°, 45°, 90°, and -45°.

The `load_panel` function also supports optional additional metadata through the `additional_metadata` parameter. This allows you to add any custom metadata to the panel beyond the standard parameters (height, width, thickness, edges_cutted).

The `additional_metadata` parameter accepts a list of dictionaries, where each dictionary must have:
- `key`: The name of the metadata field
- `value`: The value of the metadata
- `type`: The type or unit of the metadata

Examples of additional metadata for panels could include:
- Surface finish (polished, matte, etc.)
- Coating information
- Weight
- Manufacturing date
- Quality rating
- Test results

In [3]:
parent_table_name = 'materials'

parent_id_column = parent_table_name[:-1] + '_id'

parent_data = qrs.get_data(parent_table_name)

parent_data

,id_material,name_material
0,25,Prueba
1,27,Prueba 5


In [4]:
#we set the id
material_id = 25

table_name = 'panels'

In [ ]:
name = 'prueba'

height = 1
width = 2
thickness = 3

edges_cutted = False

# Define layer_layout (new parameter in load_panel function)
layer_layout = [0, 45, 90, -45]  # Example layout angles

description = 'Nachos panel 1'

# Define additional metadata
additional_metadata = [
    {'key': 'surface_finish', 'value': 'polished', 'type': 'string'},
    {'key': 'coating', 'value': 'zinc', 'type': 'string'},
    {'key': 'weight', 'value': 2.5, 'type': 'kg'}
]

# Load panel with layer_layout and additional metadata
load.load_panel(conn, name, material_id, height, width, thickness, edges_cutted, layer_layout, description, additional_metadata)

Panel 'prueba' loaded with ID: 5


5

## Make sure insert was correct

In [7]:
data = qrs.get_data_metadata(table_name)

data

,id_panel,name_panel,material_id_panel,description_panel,height_panel,width_panel,thickness_panel,edges_cutted_panel,surface_finish_panel,coating_panel,weight_panel
0,4,Prueba,25,Prueba de la web,200.0 mm,200.0 mm,5.0 mm,True bool,NaN,NaN,NaN
1,5,prueba,25,Nachos panel 1,1 mm,2 mm,3 mm,False bool,polished string,zinc string,2.5 kg
2,6,Basic Panel Example,25,Example demonstrating backward compatibility,5 mm,10 mm,1.5 mm,True bool,NaN,NaN,NaN


In [8]:
conn.close()